In [1]:
# the notebook will reload external python modules;
%load_ext autoreload
%autoreload 2

In [12]:
import sys
import os
import numpy as np
import cv2
from skimage.measure import compare_ssim as ssim

import warnings
warnings.filterwarnings("ignore")

from tqdm.autonotebook import tqdm

In [3]:
# Set parameters
FILENAME_IN =  '../data/DJI4_cam/2018-09-05/seq001/calibration/calibration.MOV'
FILENAME_OUT = '../data/DJI4_cam/2018-09-05/seq001/calibration/calibration_H264.MOV'
CODEC = 'X264' #'HEVC' 
# FC = [582.741, 580.065]  # focal lengths for GoPro Hero3+ Black
# CC = [635.154, 371.917]  # principle points for same
# KC = [-0.228, 0.0469, 0.0003, -0.0005, 0.0000] # distortion coeffs for same

In [4]:
def main(verbose=False):
    
    print('This may take a while')
    
#     cam_matrix, profile = create_matrix_profile(FC, CC, KC)

    # Load video
    video = cv2.VideoCapture(FILENAME_IN)
    
    fourcc = cv2.VideoWriter.fourcc(*list(CODEC))
    fps = video.get(cv2.CAP_PROP_FPS) 
    frame_count = video.get(cv2.CAP_PROP_FRAME_COUNT)    
    size = (int(video.get(cv2.CAP_PROP_FRAME_WIDTH)),
            int(video.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    
    writer = cv2.VideoWriter(FILENAME_OUT, fourcc, fps, size)            
    
    while video.grab() is True:
        if verbose:
            print('On frame {} of {}.'.format(video.get(cv2.CAP_PROP_POS_FRAMES), 
                                    frame_count))
#         frame =  cv2.undistort(video.retrieve()[1], cam_matrix, profile)
        frame =  video.retrieve()[1]
        writer.write(frame)
    video.release()
    writer.release()

In [5]:
main()

This may take a while


In [6]:
video = cv2.VideoCapture(FILENAME_OUT)
fourcc = cv2.VideoWriter.fourcc(*list(CODEC))
fps = video.get(cv2.CAP_PROP_FPS) 
quality = video.get(cv2.VIDEOWRITER_PROP_QUALITY) 
print(quality)

0.0


In [7]:
def compute_mse(A,B):    
    return ((A - B)**2).sum()/np.prod([*A.shape])    

In [8]:
def compare_videos(filename1, filename2, compute_every=10, verbose=False):
    
    print('this may take a while \n')
    
#     cam_matrix, profile = create_matrix_profile(FC, CC, KC)
    # Load video
    vid1 = cv2.VideoCapture(filename1)
    vid2 = cv2.VideoCapture(filename2)     
    
    fps_vid1 = vid1.get(cv2.CAP_PROP_FPS) 
    fps_vid2 = vid2.get(cv2.CAP_PROP_FPS) 
    
    frame_count_vid1 = vid1.get(cv2.CAP_PROP_FRAME_COUNT)
    frame_count_vid2 = vid2.get(cv2.CAP_PROP_FRAME_COUNT)
        
    size_vid1 = (int(vid1.get(cv2.CAP_PROP_FRAME_WIDTH)),
            int(vid1.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    
    size_vid2 = (int(vid2.get(cv2.CAP_PROP_FRAME_WIDTH)),
            int(vid2.get(cv2.CAP_PROP_FRAME_HEIGHT)))   
    
    print('Video 1:')
    print('fps: {}'.format(fps_vid1))
    print('frames: {}'.format(frame_count_vid1))
    print('resolution: {}'.format(size_vid1))    
    print('\n')
    
    print('Video 2:')
    print('fps: {}'.format(fps_vid2))
    print('frames: {}'.format(frame_count_vid2))
    print('resolution: {}'.format(size_vid2))
    print('\n')
    
    # Initializing MSE and SSIM to compute mean
    MSE_mean = 0
    SSIM_mean = 0
    
    count = 0
    n = 0
    
    while vid1.grab() is True:
        
        vid2.grab()  
        
        if verbose:
            print('On frame {} of {}.'.format(vid1.get(cv2.CAP_PROP_POS_FRAMES), 
                                              frame_count_vid1))
            print('\n')
              
#         frame =  cv2.undistort(video.retrieve()[1], cam_matrix, profile)
        frame1 = vid1.retrieve()[1]
        frame2 = vid2.retrieve()[1]        
        
        count += 1
        
        if (count % compute_every) == 0:
            
            n +=1 
            MSE = compute_mse(frame1,frame2)
            SSIM = ssim(frame1, frame2, multichannel=True) 
        
            # Cumulating MSE and SSIM to compute mean
            MSE_mean += MSE
            SSIM_mean += SSIM
        
            if verbose:
                print('MSE: {}'.format(MSE))    
                print('SSIM: {}'.format(SSIM))    
                print('\n')
        
    vid1.release()   
    vid2.release()   
    
    MSE_mean /= n
    SSIM_mean /= n
    
    return MSE_mean, SSIM_mean


In [9]:
MSE_mean, SSIM_mean = compare_videos(FILENAME_IN, FILENAME_OUT, verbose=True)

this may take a while 

Video 1:
fps: 47.952047952047955
frames: 555.0
resolution: (3840, 2160)


Video 2:
fps: 47.95
frames: 555.0
resolution: (3840, 2160)


On frame 1.0 of 555.0.


On frame 2.0 of 555.0.


On frame 3.0 of 555.0.


On frame 4.0 of 555.0.


On frame 5.0 of 555.0.


On frame 6.0 of 555.0.


On frame 7.0 of 555.0.


On frame 8.0 of 555.0.


On frame 9.0 of 555.0.


On frame 10.0 of 555.0.


MSE: 21.51626474890689
SSIM: 0.9608741778555275


On frame 11.0 of 555.0.


On frame 12.0 of 555.0.


On frame 13.0 of 555.0.


On frame 14.0 of 555.0.


On frame 15.0 of 555.0.


On frame 16.0 of 555.0.


On frame 17.0 of 555.0.


On frame 18.0 of 555.0.


On frame 19.0 of 555.0.


On frame 20.0 of 555.0.


MSE: 22.28994156700103
SSIM: 0.9592831330425599


On frame 21.0 of 555.0.


On frame 22.0 of 555.0.


On frame 23.0 of 555.0.


On frame 24.0 of 555.0.


On frame 25.0 of 555.0.


On frame 26.0 of 555.0.


On frame 27.0 of 555.0.


On frame 28.0 of 555.0.


On frame 29.0 of 555.0

On frame 259.0 of 555.0.


On frame 260.0 of 555.0.


MSE: 24.31237348894033
SSIM: 0.9533134452438997


On frame 261.0 of 555.0.


On frame 262.0 of 555.0.


On frame 263.0 of 555.0.


On frame 264.0 of 555.0.


On frame 265.0 of 555.0.


On frame 266.0 of 555.0.


On frame 267.0 of 555.0.


On frame 268.0 of 555.0.


On frame 269.0 of 555.0.


On frame 270.0 of 555.0.


MSE: 28.862350099665637
SSIM: 0.9482146470204137


On frame 271.0 of 555.0.


On frame 272.0 of 555.0.


On frame 273.0 of 555.0.


On frame 274.0 of 555.0.


On frame 275.0 of 555.0.


On frame 276.0 of 555.0.


On frame 277.0 of 555.0.


On frame 278.0 of 555.0.


On frame 279.0 of 555.0.


On frame 280.0 of 555.0.


MSE: 23.641851409786522
SSIM: 0.9540059088032754


On frame 281.0 of 555.0.


On frame 282.0 of 555.0.


On frame 283.0 of 555.0.


On frame 284.0 of 555.0.


On frame 285.0 of 555.0.


On frame 286.0 of 555.0.


On frame 287.0 of 555.0.


On frame 288.0 of 555.0.


On frame 289.0 of 555.0.


On frame 29

On frame 519.0 of 555.0.


On frame 520.0 of 555.0.


MSE: 23.943826597865225
SSIM: 0.9589455127846541


On frame 521.0 of 555.0.


On frame 522.0 of 555.0.


On frame 523.0 of 555.0.


On frame 524.0 of 555.0.


On frame 525.0 of 555.0.


On frame 526.0 of 555.0.


On frame 527.0 of 555.0.


On frame 528.0 of 555.0.


On frame 529.0 of 555.0.


On frame 530.0 of 555.0.


MSE: 23.700730332111625
SSIM: 0.9581790129793993


On frame 531.0 of 555.0.


On frame 532.0 of 555.0.


On frame 533.0 of 555.0.


On frame 534.0 of 555.0.


On frame 535.0 of 555.0.


On frame 536.0 of 555.0.


On frame 537.0 of 555.0.


On frame 538.0 of 555.0.


On frame 539.0 of 555.0.


On frame 540.0 of 555.0.


MSE: 27.483156225887345
SSIM: 0.950906472547076


On frame 541.0 of 555.0.


On frame 542.0 of 555.0.


On frame 543.0 of 555.0.


On frame 544.0 of 555.0.


On frame 545.0 of 555.0.


On frame 546.0 of 555.0.


On frame 547.0 of 555.0.


On frame 548.0 of 555.0.


On frame 549.0 of 555.0.


On frame 55

In [24]:
print('Input Video Size: {:.2f} MB'.format(os.path.getsize(FILENAME_IN)/(1024**2)))
print('Output Video Size: {:.2f} MB'.format(os.path.getsize(FILENAME_OUT)/(1024**2)))

Input Video Size: 140.64 MB
Output Video Size: 77.57 MB


In [11]:
print('MSE mean: {}'.format(MSE_mean))
print('SSIM mean: {}'.format(SSIM_mean))

MSE mean: 22.95086089263585
SSIM mean: 0.9545831155059468
